# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# Checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project. 

## Now it's time to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing the Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance at local machine (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4  

In [8]:
# Drop table before creating
query = "DROP TABLE IF EXISTS music_library"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Creating music_library table
query = "CREATE TABLE IF NOT EXISTS music_library "
# Since the session_id and the item_in_session will appear in the WHERE clause, I chose session_id \
# as the partition key, and item_in_session as the clustering column. However, there may be \
# multiple rows that have the same combination of session_id and item_in_session, so I added\
# the user_id as another clustering column. Therefore, the the sequence of the primary key is\
# session_id, item_id_session and user_id
query = query + "(artist TEXT, first_name TEXT, gender TEXT, item_in_session INT, \
                  last_name TEXT, length FLOAT, level TEXT, location TEXT, session_id INT, \
                  song TEXT, user_id INT, PRIMARY KEY (session_id, item_in_session, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'
# Insert data into music_library
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Insert data into music_library table
        query = "INSERT INTO music_library (artist, first_name, gender, item_in_session, \
                  last_name, length, level, location, session_id, song, user_id)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], \
                                float(line[5]),line[6], line[7], int(line[8]), line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT artist, song, length FROM music_library WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

result_1 = []
for row in rows:
    #print (row.artist, row.song, row.length)
    result_1.append(row)

# Create a dataframe to beautify the output
df_1 = pd.DataFrame(result_1, columns = ['artist', 'song', 'length'])
print(df_1)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


The result shows that only one record meet the requirement that the seesion_id is 338 and the item_in_session
is 4, and the corresponding values are Faithless, Music Matters (Mark Knight Dub) and 495.307312

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
query = "DROP TABLE IF EXISTS song_plays"

try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [13]:
query = "CREATE TABLE IF NOT EXISTS song_plays "
# Similar to query one, since the user_id and session_id will appear at the WHERE clause,\
# the user_id is chosen to be the parition key, the session_id is set to be the clustering \
# column, and the addtional clustering column is item_in_session.
query = query + "(artist TEXT, first_name TEXT, gender TEXT, item_in_session INT, \
                  last_name TEXT, length FLOAT, level TEXT, location TEXT, session_id INT, \
                  song TEXT, user_id INT, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # Insert data into song_plays table
        query = "INSERT INTO song_plays (artist, first_name, gender, item_in_session, \
                  last_name, length, level, location, session_id, song, user_id)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4],\
                                float(line[5]),line[6], line[7], int(line[8]), line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
# Verify the data was entered into the table
query = "SELECT artist, song, item_in_session, first_name, last_name\
         FROM song_plays WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

result_2 = []
for row in rows:
    result_2.append(row)
    #print (row.artist, row.song, row.first_name, row.last_name, row.item_in_session)

# Create a dataframe to beautify the output
df_2 = pd.DataFrame(result_2, columns = ['artist', 'song', 'item_in_session', 'first_name', 'last_name'])
pd.set_option('display.width', 500)
print(df_2)

              artist                                               song  item_in_session first_name last_name
0   Down To The Bone                                 Keep On Keepin' On                0     Sylvie      Cruz
1       Three Drives                                        Greece 2000                1     Sylvie      Cruz
2  Sebastien Tellier                                          Kilometer                2     Sylvie      Cruz
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...                3     Sylvie      Cruz


The are four records that both have user_id equal to 10 and session_id equal to 182.

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
# Drop table before creating
query = "DROP TABLE IF EXISTS music_history"

try:
    session.execute(query)
except Exception as e:
    print(e)             

In [18]:
#Craete music_history table
query = "CREATE TABLE IF NOT EXISTS music_history "
# Since one song may be listened by different users, the user_id is needed as the clustering\
# column to uniquely identify each row. Therefore, the primay key is made up of song, the \
# partition key, and user_id, the clustering column.
query = query + "(artist TEXT, first_name TEXT, gender TEXT, item_in_session INT, \
                  last_name TEXT, length FLOAT, level TEXT, location TEXT, session_id INT, \
                  song TEXT, user_id INT, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Insert data into music_history table
        query = "INSERT INTO music_history (artist, first_name, gender, item_in_session, \
                  last_name, length, level, location, session_id, song, user_id)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4],\
                                float(line[5]),line[6], line[7], int(line[8]), line[9], int(line[10])))

In [20]:
## Verify the data was entered into the table
query = "SELECT first_name, last_name\
         FROM music_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

result_3 = []
for row in rows:
    result_3.append(row)
    #print (row.first_name, row.last_name)

# Create a dataframe to beautify the output
df_3 = pd.DataFrame(result_3, columns = ['first_name', 'last_name'])
#pd.set_option('display.width', 500)
print(df_3)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


Three users, Jacqueline Lynch, Tegan Levine and Sara Johnson, listened to the song 'All Hands Against His Own'.

### Drop the tables before closing out the sessions

In [62]:
query = "DROP TABLE IF EXISTS music_library"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_plays"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS music_history"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [63]:
session.shutdown()
cluster.shutdown()